# Part 1 - loading the Toronto data into dataframe

In [1]:
import pandas as pd
import numpy as np

In [2]:
toronto_neighborhoods = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', match = 'Neighbourhood') # read table with postal code, borough, and neighborhood into list of tables
print("Number of tables in list: ",len(toronto_neighborhoods)) #check that only one table has been loaded
toronto_neighborhoods=toronto_neighborhoods[0] #convert to dataframe

Number of tables in list:  1


In [3]:
toronto_neighborhoods.head() #Confirm that data is in dataframe
toronto_neighborhoods.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)

In [4]:
t_hoods = toronto_neighborhoods[toronto_neighborhoods['Borough']!= 'Not assigned'].reset_index(drop=True) #remove postal codes with "not assigned" as borough
t_hoods.head(10) #visualize head to confirm that appropriate rows were removed
# t_hoods.shape

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


**Now we need to find rows that have "not assigned" neighbourhoods and rename them to match Borough name.**

In [5]:
for neighborhood in t_hoods['Neighborhood']:
    if 'Not assigned' in neighborhood: #assume that if a neighborhood is unassigned, then 'not assigned' will not be part of a list of neighborhoods
        t_hoods[neighborhood] = t_hoods['Borough']
t_hoods.groupby('Postal Code')
# t_hoods.sort_values('Postal Code')
t_hoods.shape #show shape of dataframe

(103, 3)

----
# Part 2 - Adding latitude and longitude to the dataframe

In [6]:
#Geocoder didn't work, so we'll use the csv data
!wget -q -O 'Geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded')
lat_lng_coords = pd.read_csv('Geospatial_data.csv')
# with open('Geospatial_data.csv') as gps_data:
#     lat_lng_coords = csv.load(gps_data)
lat_lng_coords

Data downloaded


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [7]:
#Change the index so that we can merge columns from lat_lng_coords into the t_hoods dataframe
t_hoods.set_index('Postal Code', inplace=True)
print(t_hoods.head())
lat_lng_coords.set_index('Postal Code', inplace=True)
lat_lng_coords.head()

                      Borough                                 Neighborhood
Postal Code                                                               
M3A                North York                                    Parkwoods
M4A                North York                             Victoria Village
M5A          Downtown Toronto                    Regent Park, Harbourfront
M6A                North York             Lawrence Manor, Lawrence Heights
M7A          Downtown Toronto  Queen's Park, Ontario Provincial Government


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [8]:
#Copy data over to t_hoods, verify that the coordinates are correct
t_hoods['Latitude']=lat_lng_coords['Latitude']
t_hoods['Longitude']=lat_lng_coords['Longitude']
t_hoods.reset_index(inplace=True)
t_hoods[t_hoods['Postal Code']=='M1B']

,Postal Code,Borough,Neighborhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353


In [9]:
print("Shape of dataframe: ",t_hoods.shape)
print(t_hoods.head())

Shape of dataframe:  (103, 5)
  Postal Code           Borough                                 Neighborhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -79.464763  
4  43.662301 -79.389494  


# Part 3 - Clustering Toronto neighborhoods

In [10]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip -q install folium # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [11]:
toronto_proper = t_hoods[(t_hoods['Borough'].str.contains('Toronto'))] # Keep all rows where 'Toronto' appears in the borough name and assign to a new dataframe called toronto_proper
toronto_proper.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


**Create a map to visualize neighborhood locations, verifying that our data processing has worked as expected.**

In [12]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="tdot_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_proper['Latitude'], toronto_proper['Longitude'], toronto_proper['Borough'], toronto_proper['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

**Beautiful.  Dot markers appear for neighborhoods in Toronto proper, and not the other boroughs, like Scarborough.  Now it's time to work with the Foursquare data.**

In [52]:
# The code was removed by Watson Studio for sharing.

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_venues = getNearbyVenues(names = toronto_proper['Neighborhood'], latitudes = toronto_proper['Latitude'],longitudes = toronto_proper['Longitude'])
toronto_venues_backup=toronto_venues
toronto_venues_backup.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


Let's remove all neighborhoods that contain fewer than 10 venues, since such neighborhoods will be difficult to cluster.

In [29]:
venue_count = toronto_venues_backup.groupby('Neighborhood').count()
venue_count
# toronto_venues_large = venue_count[(venue_count['Venue']>9)]
# toronto_venues_large.reset_index(inplace=True)
# toronto_venues.reset_index(inplace=True)
# toronto_venues_sorted = pd.merge(left=toronto_venues, right=toronto_venues_large, left_on = 'Neighborhood', right_on = 'Neighborhood')
# print(toronto_venues.shape)
# print(toronto_venues_sorted.shape)
# toronto_venues_sorted.set_index('Neighborhood')
# toronto_venues_sorted.rename(columns ={'Neighborhood Latitude_x':'Neighborhood Latitude',\
#                                        'Neighborhood Longitude_x':'Neighborhood Longitude',\
#                                        'Venue_x':'Venue','Venue Latitude_x':'Venue Latitude',\
#                                        'Venue Longitude_x':'Venue Longitude',\
#                                        'Venue Category_x': 'Venue Category'},inplace=True)

,index,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
Berczy Park,100,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",85,85,85,85,85,85,85
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",44,44,44,44,44,44,44
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",26,26,26,26,26,26,26
Central Bay Street,100,100,100,100,100,100,100
Christie,25,25,25,25,25,25,25
Church and Wellesley,100,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100,100
Davisville,59,59,59,59,59,59,59


In [20]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues_sorted[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues_sorted['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store

In [21]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,College Theater,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Social Club,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Tram Station,Tunnel,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store

In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Café,Restaurant,Beer Bar,Japanese Restaurant,Park,Farmers Market,Bakery,Cocktail Bar
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Café,Nightclub,Bar,Restaurant,Supermarket,Breakfast Spot,Sandwich Place,Furniture / Home Store,Performing Arts Venue
2,"Business reply mail Processing Centre, South C...",Fast Food Restaurant,Park,Brewery,Harbor / Marina,Coffee Shop,Restaurant,Light Rail Station,Bakery,Burrito Place,Gym / Fitness Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Rental Car Location,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Plane,Bar
4,Central Bay Street,Coffee Shop,Café,Art Gallery,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Hotel,Breakfast Spot,Bubble Tea Shop,Sandwich Place


In [43]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

In [32]:
kmeans.labels_[0:len(kmeans.labels_)]

array([0, 1, 1, 4, 0, 2, 0, 0, 1, 3, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1], dtype=int32)

In [44]:
toronto_proper_sorted = pd.merge(left=toronto_proper, right=toronto_venues_large, left_on = 'Neighborhood', right_on = 'Neighborhood') # remove unused neighborhoods (those with less than 10 venues)

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_proper_sorted

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

print(toronto_merged.head()) # check the last columns!

  Postal Code           Borough                                 Neighborhood  \
0         M5A  Downtown Toronto                    Regent Park, Harbourfront   
1         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
2         M5B  Downtown Toronto                     Garden District, Ryerson   
3         M5C  Downtown Toronto                               St. James Town   
4         M4E      East Toronto                                  The Beaches   

    Latitude  Longitude  Neighborhood Latitude  Neighborhood Longitude  Venue  \
0  43.654260 -79.360636                     71                      71     71   
1  43.662301 -79.389494                     62                      62     62   
2  43.657162 -79.378937                    100                     100    100   
3  43.651494 -79.375418                    100                     100    100   
4  43.676357 -79.293031                     39                      39     39   

   Venue Latitude  Venue Longitu

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Now let's looks at the makeup of the different clusters.**

As you can see, cluster 0 is located in Downtown Toronto and is predominated by coffee shops and hotels.  Cluster 1 is geographically diverse and is dominated by pubs, coffee shops, and restaurants.  The remaining three clusters feature fewer dining options than cluster 0 and 1 and are differentiated from each other by their non-food options.  For example, cluster 4 has many airport services, which do not appear in the top 10 for the other clusters.

In [46]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,71,71,71,71,71,71,0,Coffee Shop,Park,Café,Bakery,Restaurant,Pub,Theater,Breakfast Spot,Performing Arts Venue,Thai Restaurant
1,Downtown Toronto,62,62,62,62,62,62,0,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Park,Yoga Studio,Portuguese Restaurant,Pharmacy,Pizza Place,College Theater
2,Downtown Toronto,100,100,100,100,100,100,0,Coffee Shop,Hotel,Gastropub,Sushi Restaurant,Japanese Restaurant,Plaza,Ramen Restaurant,Theater,Department Store,Restaurant
3,Downtown Toronto,100,100,100,100,100,100,0,Coffee Shop,Café,Seafood Restaurant,Restaurant,Gastropub,Beer Bar,Bakery,Hotel,American Restaurant,Clothing Store
5,Downtown Toronto,100,100,100,100,100,100,0,Coffee Shop,Hotel,Café,Restaurant,Beer Bar,Japanese Restaurant,Park,Farmers Market,Bakery,Cocktail Bar
6,Downtown Toronto,100,100,100,100,100,100,0,Coffee Shop,Café,Art Gallery,Japanese Restaurant,Italian Restaurant,Middle Eastern Restaurant,Hotel,Breakfast Spot,Bubble Tea Shop,Sandwich Place
8,Downtown Toronto,100,100,100,100,100,100,0,Coffee Shop,Café,Hotel,Restaurant,Theater,Bar,Clothing Store,Cosmetics Shop,American Restaurant,Sushi Restaurant
10,Downtown Toronto,99,99,99,99,99,99,0,Coffee Shop,Hotel,Boat or Ferry,Restaurant,Plaza,Brewery,Baseball Stadium,Café,Park,Music Venue
13,Downtown Toronto,100,100,100,100,100,100,0,Hotel,Coffee Shop,Café,Restaurant,Gym,Seafood Restaurant,American Restaurant,Plaza,Japanese Restaurant,Italian Restaurant
16,Downtown Toronto,100,100,100,100,100,100,0,Coffee Shop,Hotel,Café,Japanese Restaurant,Gastropub,Restaurant,Beer Bar,American Restaurant,Seafood Restaurant,Concert Hall


In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,39,39,39,39,39,39,1,Pub,Coffee Shop,Health Food Store,Bar,Breakfast Spot,Sandwich Place,Pizza Place,Bakery,Café,Cheese Shop
9,West Toronto,24,24,24,24,24,24,1,Gym,Pharmacy,Coffee Shop,Park,Bakery,Discount Store,Italian Restaurant,Brewery,Supermarket,Smoke Shop
11,West Toronto,90,90,90,90,90,90,1,Bar,Café,Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Cocktail Bar,Theater,Bakery,Asian Restaurant,Men's Store
12,East Toronto,90,90,90,90,90,90,1,Greek Restaurant,Coffee Shop,Café,Pub,Grocery Store,Fast Food Restaurant,Italian Restaurant,Yoga Studio,Bank,Bakery
14,West Toronto,85,85,85,85,85,85,1,Coffee Shop,Café,Nightclub,Bar,Restaurant,Supermarket,Breakfast Spot,Sandwich Place,Furniture / Home Store,Performing Arts Venue
15,East Toronto,51,51,51,51,51,51,1,Indian Restaurant,Park,Grocery Store,Gym,Fast Food Restaurant,Coffee Shop,Restaurant,Italian Restaurant,Sandwich Place,Café
17,East Toronto,77,77,77,77,77,77,1,Coffee Shop,Café,American Restaurant,Sandwich Place,Bar,Bakery,Diner,Vietnamese Restaurant,Brewery,Italian Restaurant
19,West Toronto,61,61,61,61,61,61,1,Café,Bar,Italian Restaurant,Bakery,Thai Restaurant,Grocery Store,Park,Antique Shop,Mexican Restaurant,Flea Market
20,Central Toronto,39,39,39,39,39,39,1,Sporting Goods Shop,Café,Coffee Shop,Italian Restaurant,Restaurant,Clothing Store,Diner,Cosmetics Shop,Spa,Salon / Barbershop
21,Central Toronto,57,57,57,57,57,57,1,Coffee Shop,Park,Café,Sandwich Place,Pub,Pizza Place,Burger Joint,Vegetarian / Vegan Restaurant,Diner,Pharmacy


In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,25,25,25,25,25,25,2,Grocery Store,Café,Park,Coffee Shop,Playground,Storage Facility,Beer Store,Candy Store,Nightclub,Bakery


In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,21,21,21,21,21,21,3,Dog Run,Park,Thai Restaurant,Dance Studio,Brewery,Food & Drink Shop,Sushi Restaurant,Café,Taco Place,Bar


In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Downtown Toronto,26,26,26,26,26,26,4,Rental Car Location,Sculpture Garden,Harbor / Marina,Boat or Ferry,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Plane,Bar
